In [13]:
import numpy as np
import cv2 as cv
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from PIL import Image
import imghdr
from keras.preprocessing import image

In [14]:
cnn = load_model('model.h5')

In [15]:
def detect_faces (frame):
    (h, w) = frame.shape[:2]
    prototxtPath = r"deploy.prototxt"
    weightsPath = r"res10_300x300_ssd_iter_140000.caffemodel"
    faceNet = cv.dnn.readNet(prototxtPath,weightsPath)
    blob = cv.dnn.blobFromImage(frame, 1.0, (224,224),(104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    faces = []
    locs = []
    preds = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            face = frame[startY:endY, startX:endX]
            locs.append((startX, startY, endX, endY))
            return face,locs

In [16]:
capture = cv.VideoCapture(0)
while True:
    isTrue, frame = capture.read()
    framee,loc = detect_faces(frame)
    framee = cv.resize(framee,(64,64))
    framee = np.expand_dims(framee, axis = 0)
    number = cnn.predict(framee)
    if(number>0.5):
        prediction = 'Mask'
        cv.putText(frame,prediction,(loc[0][0],loc[0][1]),cv.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
        cv.rectangle(frame, (loc[0][0],loc[0][1]), (loc[0][2],loc[0][3]), (0,255,0), 2)
    else:
        prediction = 'NO Mask'
        cv.putText(frame,prediction,(loc[0][0],loc[0][1]),cv.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        cv.rectangle(frame, (loc[0][0],loc[0][1]), (loc[0][2],loc[0][3]), (0,0,255), 2)
    
    cv.imshow('Video',frame)
    if cv.waitKey(20) & 0xFF==ord('d'):
        cv.destroyAllWindows()
        break
capture.release()